Basic Mechanics
-------------------------

Let's get an overivew of how authentication/etc work with Planet's library and set the groundwork for how we're going to interact with the Planet Data API.

Note that we're trying to stay relatively language agnostic, so we're still going to build things up mostly from scratch.  We'll make use of Python's standard library functionality and we'll use the `requests` library to simplify communicating with the api a bit.  We're not going to focus heavily on Planet's Python client library or other high-level ways of accessing the data.  Our goal here is to explore and understand the underlying REST api.

To start with, let's make a request against the top-level endpoint for the Planet data api. It returns some information about the api and doesn't require authentication (we'll cover that soon).


In [ ]:
# Just for nicer display of response dicts -- This is a standard library module
from pprint import pprint  

# Not part of the standard library, but makes life much easier when working with http APIs
import requests

# Let's do this properly and raise an exception if we don't get a 200 (ok) response code
response = requests.get('https://api.planet.com/data/v1')
response.raise_for_status()
body = response.json()

# Nicer display than a bare print
pprint(body) 

Authentication
----------------------

Planet's APIs handle authentication primarily through API keys. It's a bit more secure than using your username and password, as the key only has access to our api (and not, say, deleting your account).  

You can find your api key by logging in to https://www.planet.com/account  You should already have had an account created for you as part of the workshop signup.  Your account will have access to a variety of data in California for the next month.

The python client reads from the `PL_API_KEY` environment variable if it exists. We'll read the api key from that env variable rather than hard-coding it in these exercises.  Let's go ahead and set that now:


In [ ]:
import os

# If you're following along with this notebook, you can enter your API Key on the following line, and uncomment it:
#os.environ['PL_API_KEY'] = "YOUR API KEY HERE"

# Setup the API Key from the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')
assert PLANET_API_KEY is not None, 'PL_API_KEY not found'

To start with, let's look at what happens when we don't authenticate:

In [ ]:
# We'll explain this url later -- This is just a demo of something you can't access without auth
scene_url = 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20191010_183406_0f28'

response = requests.get(scene_url)
response.raise_for_status()

You can use your api key to authenticate through basic http authentication:

In [ ]:
# Basic auth for our API expects api key as the username and no password
response = requests.get(scene_url, auth=(PLANET_API_KEY, ''))
response.raise_for_status()

We can also use a session in the `requests` library that will store our auth and use it for all requests:

In [ ]:
session = requests.Session()
session.auth = (PLANET_API_KEY, '')

response = session.get(scene_url)
response.raise_for_status()

Rate Limiting and Retries
--------------------------------------

Planet's services limit the number of requests you can make in a short amount of time to avoid unintentional and intentional denial of service attacks.  A 429 response code indicates that the service won't deliver results until you slow down.

As an example, let's do something that will trigger a "slow down" response (429):

In [ ]:
# We'll use multiple threads to really hammer things... Otherwise it's unlikely to trigger.
from concurrent.futures import ThreadPoolExecutor

def make_request(ignored):
    requests.get(scene_url, auth=(PLANET_API_KEY, '')).raise_for_status()
    
nthreads = 8
with ThreadPoolExecutor(nthreads) as executor:
    for _ in executor.map(make_request, range(100)):
        pass

We ask that you follow an exponential backoff pattern when this occurs -- E.g. on the first occurence, wait `n` seconds, on the second `n**2` seconds, on the third `n**3`, etc.

I'm sure a lot of you have probably implemented similar retry functionality or have a library you often use to do so.  There are also methods for this in our python client.

However, to keep with the spirit of staying relatively low-level and using widely available tools, we'll set up the `requests` library to do this for us.

In [ ]:
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

session = requests.Session()
session.auth = (PLANET_API_KEY, '')

retries = Retry(total=5,
                backoff_factor=0.2, # Exponential backoff scaled by 0.2
                status_forcelist=[429]) # In practice, you may want other codes there too, e.g. 500s...

session.mount('https://', HTTPAdapter(max_retries=retries))

And just to demonstrate that we can now hammer the service in parallel and properly back off and retry when it asks us to:

In [ ]:
def make_request(ignored):
    # Note that we're using the session we set up above
    session.get(scene_url, auth=(PLANET_API_KEY, '')).raise_for_status()
    
nthreads = 8
with ThreadPoolExecutor(nthreads) as executor:
    for _ in executor.map(make_request, range(100)):
        pass

Setup for Future Exercises
----------------------------------------

Taking what we've just walked through, we'll use the following as a bit of boilerplate to set up our later exercises.  For actual use cases, you can see how this could form the start of an `APIClient` class. We're going to keep things as mininimal as possible in this tutorial, though.

In [ ]:
import os
from pprint import pprint
from urllib3.util.retry import Retry

import requests
from requests.adapters import HTTPAdapter

PLANET_API_URL = 'https://api.planet.com/data/v1'

def setup_session(api_key=None):
    """
    Initialize a requests.Session that handles Planet api key auth and retries.
    
    :param str api_key:
        A Planet api key. Will be read from the PL_API_KEY env var if not specified.
    
    :returns requests.Session session:
        A Session instance optimized for use with Planet's api.
    """
    if api_key is None:
        api_key = os.getenv('PL_API_KEY')

    session = requests.Session()
    session.auth = (api_key, '')

    retries = Retry(total=5,
                    backoff_factor=0.2,  
                    status_forcelist=[429])

    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

session = setup_session() # Or pass in an api key if the environment variable isn't set